In [ ]:
# copied from
# https://github.com/Yelp/yelp-fusion/blob/master/fusion/python/sample.py


from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib


# This client code can run on Python 2.x or 3.x.  Your imports can be
# simpler if you only need one of those.
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode


# OAuth credential placeholders that must be filled in by users.
# You can find them on
# https://www.yelp.com/developers/v3/manage_app
CLIENT_ID = None # Insert yours here!
CLIENT_SECRET = None # Insert yours here!

# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.
TOKEN_PATH = '/oauth2/token'
GRANT_TYPE = 'client_credentials'


# Defaults for our simple example.
DEFAULT_TERM = 'dinner'
DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 3


def obtain_bearer_token(host, path):
    """Given a bearer token, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        str: OAuth bearer token, obtained using client_id and client_secret.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    assert CLIENT_ID, "Please supply your client_id."
    assert CLIENT_SECRET, "Please supply your client_secret."
    data = urlencode({
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'grant_type': GRANT_TYPE,
    })
    headers = {
        'content-type': 'application/x-www-form-urlencoded',
    }
    response = requests.request('POST', url, data=data, headers=headers)
    bearer_token = response.json()['access_token']
    return bearer_token


def request(host, path, bearer_token, url_params=None):
    """Given a bearer token, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        bearer_token (str): OAuth bearer token, obtained using client_id and client_secret.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % bearer_token,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


In [ ]:
token = obtain_bearer_token(API_HOST, TOKEN_PATH)


In [ ]:
import time
import pandas as pd
X = pd.read_csv('/Users/adamkelleher/Downloads/CPC990AllData.csv')

In [ ]:
phone_info = {}
for phone in X['ORGBOOKPHONE']:
    if phone:
        try:
            number = '+1{}'.format(int(phone))
            path = '/v3/businesses/search/phone'
            phone_info[phone] = request(API_HOST, path, token, url_params={'phone': number})
            print(phone_info[phone])
        except:
            pass


In [ ]:
{k: v for k, v in phone_info.items() if v.get('total', 0) > 0}

In [ ]:
phone_info = {}
for phone in X['ORGBOOKPHONE']:
    if phone:
        try:
            number = '+1{}'.format(int(phone))
            path = '/v3/businesses/search/phone'
            phone_info[phone] = request(API_HOST, path, token, url_params={'phone': number})
            print(phone_info[phone])
        except:
            pass

In [ ]:
location_info = {}
for i, (location, ein) in X[['LOCATION', 'EIN']].iterrows():
    if location:
        try:
            path = '/v3/businesses/search'
            location_info[ein] = request(API_HOST, path, token, url_params={'location': location})
        except:
            pass

In [ ]:
X['LOCATION'] = X['ADDRESS'] + ', ' + X['CITY'] + ', ' + X['STATE'] + ', ' + X['ZIP'].apply(str)

In [ ]:
for i, (location, ein) in X[['LOCATION', 'EIN']].iterrows():
    pass

In [ ]:
location_info.values()[:10]

In [ ]:
location = '4 GINGER CREEK PARKWAY, GLEN CARBON, IL, 62034'
path = '/v3/businesses/search'
request(API_HOST, path, token, url_params={'location': location})